In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import json
import glob

#gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel

#spacy
import spacy

#vis
import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

c:\Users\vitor\AppData\Local\Programs\Python\Python310\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
C:\Users\vitor\AppData\Roaming\Python\Python310\site-packages\joblib\backports.py:22: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils  # noqa


In [2]:
df = pd.read_csv('../oportunidades.csv')
data = list(df['opo_texto'])

In [3]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    return [" ".join([token.lemma_ for token in nlp(text) if token.pos_ in allowed_postags]) for text in texts]

lemmatized_texts = lemmatization(data)

In [4]:
def gen_words(texts):
    return [gensim.utils.simple_preprocess(text, deacc=True) for text in texts]

data_words = gen_words(lemmatized_texts)

In [5]:
#BIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

data_bigrams = make_bigrams(data_words)

In [6]:
import itertools

id2word = corpora.Dictionary(data_bigrams)
texts = data_bigrams
corpus = [id2word.doc2bow(text) for text in texts]

c_words = list(itertools.chain.from_iterable(corpus))
print('numero de palavras no corpus = ',len(c_words))

numero de palavras no corpus =  93651


In [7]:
#DF FILTER
id2word = corpora.Dictionary(data_bigrams)
texts = data_bigrams
corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value=0.4
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    tfidf_ids = [id for id,value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    hfreq_words = [id for id in bow_ids if tfidf.dfs[id]/len(corpus) > low_value]
    drops = hfreq_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in hfreq_words and b[0] not in words_missing_in_tfidf]       
    corpus[i] = new_bow

In [8]:
#TF-IDF FILTER
id2word = corpora.Dictionary(data_bigrams)
texts = data_bigrams
corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value=0.02
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]       
    corpus[i] = new_bow

# MODEL MULTICORE

In [9]:
K = 20
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=K,
                                           random_state=100,
                                           chunksize=100,
                                           passes=10)

lda_model.save('models/lda_model.model')

# Visualization

In [11]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=K)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.004976 -0.029484       1        1  22.890244
3      0.305813 -0.063062       2        1  12.301525
6      0.137557 -0.185988       3        1   9.510463
16     0.134771 -0.017224       4        1   7.377515
17     0.278106 -0.157843       5        1   5.945995
5      0.005458 -0.140574       6        1   5.625837
19    -0.173924 -0.209239       7        1   5.543676
13     0.312233  0.046138       8        1   5.065993
1      0.126120  0.353616       9        1   3.669017
7     -0.109893 -0.246919      10        1   3.628462
8      0.041928 -0.285954      11        1   3.602087
11     0.234974  0.133736      12        1   3.132964
0     -0.127089  0.313996      13        1   2.451544
18    -0.237334  0.093337      14        1   1.997285
4     -0.044647  0.076439      15        1   1.662974
14    -0.207297  0.215737      16        1   1.597523
2     -0.317963 -0.009845      17        1   1.537453
10    -0.140526  0.037662      18        1   1.296632
12     0.078661  0.179350      19        1   0.798193
15    -0.291972 -0.103880      20        1   0.364617, topic_info=             Term         Freq        Total Category  logprob  loglift
92        project  1581.000000  1581.000000  Default  20.0000  20.0000
5     application  2338.000000  2338.000000  Default  19.0000  19.0000
101      research  2290.000000  2290.000000  Default  18.0000  18.0000
1711   nomination   382.000000   382.000000  Default  17.0000  17.0000
51          grant   769.000000   769.000000  Default  16.0000  16.0000
...           ...          ...          ...      ...      ...      ...
57        include     2.083642   613.191529  Topic20  -5.5102  -0.0705
9            base     2.079396   323.395123  Topic20  -5.5122   0.5673
4       applicant     2.077997   759.976219  Topic20  -5.5129  -0.2878
60          issue     2.058954    94.458273  Topic20  -5.5221   1.7881
5     application     2.057683  2338.039573  Topic20  -5.5227  -1.4214

[990 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
5049     17  0.622780  abundant
4656      1  0.055615     ac_uk
4656     10  0.889836     ac_uk
122       1  0.034173  academic
122       2  0.606563  academic
...     ...       ...       ...
705       7  0.206127     young
705       8  0.008962     young
705       9  0.017924     young
705      14  0.206127     young
4035     16  0.796523      zone

[3230 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 4, 7, 17, 18, 6, 20, 14, 2, 8, 9, 12, 1, 19, 5, 15, 3, 11, 13, 16])